In [1]:
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# config
num_trading_days_in_year = 252

In [26]:
df_fund = pd.read_csv("../data/Parag Parikh Flexi Cap Fund - Direct Plan - Growth.csv", index_col="date", parse_dates=True, date_format="%d-%m-%Y")
df_fund

,nav,dayChange
date,,
2022-11-09,51.7279,0.2760
2022-11-07,51.4519,0.4838
2022-11-04,50.9681,-0.3444
2022-11-03,51.3125,-0.3929
2022-11-02,51.7054,-0.2795
...,...,...
2013-06-03,10.0572,0.0418
2013-05-31,10.0154,-0.0173
2013-05-30,10.0327,0.0247


In [27]:
# sort df_nav
df_fund = df_fund[::-1]

df_fund.index

DatetimeIndex(['2013-05-28', '2013-05-29', '2013-05-30', '2013-05-31',
               '2013-06-03', '2013-06-04', '2013-06-05', '2013-06-06',
               '2013-06-07', '2013-06-10',
               ...
               '2022-10-25', '2022-10-27', '2022-10-28', '2022-10-31',
               '2022-11-01', '2022-11-02', '2022-11-03', '2022-11-04',
               '2022-11-07', '2022-11-09'],
              dtype='datetime64[ns]', name='date', length=2325, freq=None)

In [30]:
def absolute_return(initial_value, final_value):
  return (final_value - initial_value) * 100 / initial_value

In [31]:
def one_day_absolute_return(df_nav):
  return absolute_return(df_nav.iloc[-2]["nav"], df_nav.iloc[-1]["nav"])

one_day_absolute_return(df_fund)

0.5364233390797934

In [32]:
def one_year_absolute_return(df_nav):
  return absolute_return(df_nav.iloc[-1-num_trading_days_in_year]["nav"], df_nav.iloc[-1]["nav"])

one_year_absolute_return(df_fund)

-2.66428068749318

In [33]:
# generic function

def n_years_absolute_return(df_nav, n_years=7):
  num_days = num_trading_days_in_year*n_years
  return absolute_return(df_nav.iloc[-1-num_days]["nav"], df_nav.iloc[-1]["nav"])

In [34]:
for n_years in (1, 3, 5, 7):
  return_ = n_years_absolute_return(df_fund, n_years=n_years)
  print("Years:", n_years, "\t", "Abs. Return:", return_)

Years: 1 	 Abs. Return: -2.66428068749318
Years: 3 	 Abs. Return: 93.95247165396843
Years: 5 	 Abs. Return: 132.60830462897178
Years: 7 	 Abs. Return: 224.50816792552254


In [35]:
def trailing_return(initial_value, final_value, n_years):
  return (((final_value / initial_value) ** (1 / n_years)) - 1)*100

In [36]:
def n_years_trailing_return(df_nav, n_years):
  num_days = num_trading_days_in_year*n_years

  final_value = df_nav.iloc[-1]["nav"]
  initial_value = df_nav.iloc[-1-num_days]["nav"]

  return trailing_return(initial_value, final_value, n_years)

In [37]:
for n_years in (1, 3, 5, 7):
  return_ = n_years_trailing_return(df_fund, n_years=n_years)
  print("Years:", n_years, "\t", "Trl. Return:", return_)

Years: 1 	 Trl. Return: -2.6642806874931813
Years: 3 	 Trl. Return: 24.70918455522999
Years: 5 	 Trl. Return: 18.392732230259657
Years: 7 	 Trl. Return: 18.3129364010578


In [53]:
def n_years_absolute_rolling_returns(df_nav, n_years):
  df_rr = df_nav["nav"].rolling(num_trading_days_in_year*n_years, closed="both").apply(lambda window: absolute_return(window.iloc[0], window.iloc[-1])).dropna()
  df_rr.rename("rolling_returns", inplace=True)
  return df_rr

In [54]:
n_years_absolute_rolling_returns(df_fund, 5)

date
2018-07-10    153.677294
2018-07-11    153.084247
2018-07-12    152.890687
2018-07-13    152.431549
2018-07-16    151.282026
                 ...    
2022-11-02    130.059444
2022-11-03    127.534754
2022-11-04    129.015560
2022-11-07    131.872896
2022-11-09    132.608305
Name: rolling_returns, Length: 1066, dtype: float64

In [55]:
def n_years_trailing_rolling_returns(df_nav, n_years):
  df_rr = df_nav["nav"].rolling(num_trading_days_in_year*n_years, closed="both").apply(lambda window: trailing_return(window.iloc[0], window.iloc[-1], n_years)).dropna()
  df_rr.rename("rolling_returns", inplace=True)
  return df_rr

In [56]:
n_years_trailing_rolling_returns(df_fund, 5)

date
2018-07-10    20.463734
2018-07-11    20.407357
2018-07-12    20.388934
2018-07-13    20.345187
2018-07-16    20.235381
                ...    
2022-11-02    18.132124
2022-11-03    17.871700
2022-11-04    18.024725
2022-11-07    18.317776
2022-11-09    18.392732
Name: rolling_returns, Length: 1066, dtype: float64

In [57]:
df_rr_three = n_years_trailing_rolling_returns(df_fund, 3)
df_rr_three.loc["2022-11-02"]

26.01046704050316

In [58]:
fig = px.line(df_rr_three, x=df_rr_three.index, y="rolling_returns", title="Rolling 3-year returns")
fig.show()

/home/kadam/miniconda3/envs/mutual-fund-analysis/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

